In [1]:
import pandas as pd
import csv
import re
import statsmodels.api as sm


from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
file_url = '/Users/xzhou/github/project_archives/files_airbnb/sc_airbnb/{}.csv'

listings_file = file_url.format('listings_details')
reviews_file = file_url.format('reviews_details')
calendar_file = file_url.format('calendar_details')

In [3]:
file_url = '/Users/xzhou/github/project_archives/files_airbnb/{}.csv'

sf_file = file_url.format('sf_airbnb/listings_details')
ok_file = file_url.format('ok_airbnb/listings_details')
sc_file = file_url.format('sc_airbnb/listings_details')
scz_file = file_url.format('scz_airbnb/listings_details')

In [4]:
date_to_parse = ['last_scraped','host_since', 'first_review', 'last_review']

listings_sf = pd.read_csv(sf_file, parse_dates=date_to_parse)
listings_ok = pd.read_csv(ok_file, parse_dates=date_to_parse)
listings_sc = pd.read_csv(sc_file, parse_dates=date_to_parse)
listings_scz = pd.read_csv(scz_file, parse_dates=date_to_parse)

In [5]:
print(listings_sf.shape, listings_ok.shape, listings_sc.shape, listings_scz.shape)

(6633, 96) (2898, 96) (5668, 96) (1570, 96)


In [6]:
# Created keys for future references

frames = [listings_sf, listings_ok, listings_sc, listings_scz]
keys = ['listings_sf', 'listings_ok', 'listings_sc', 'listings_scz']

In [7]:
df_listings = pd.concat(frames, keys=keys)
df_listings.head()

id                        listing_url     scrape_id  \
listings_sf 0   958   https://www.airbnb.com/rooms/958  2.018080e+13   
            1  5858  https://www.airbnb.com/rooms/5858  2.018080e+13   
            2  7918  https://www.airbnb.com/rooms/7918  2.018080e+13   
            3  8142  https://www.airbnb.com/rooms/8142  2.018080e+13   
            4  8339  https://www.airbnb.com/rooms/8339  2.018080e+13   

              last_scraped                                               name  \
listings_sf 0   2018-08-06                Bright, Modern Garden Unit - 1BR/1B   
            1   2018-08-06                                 Creative Sanctuary   
            2   2018-08-06         A Friendly Room - UCSF/USF - San Francisco   
            3   2018-08-06  Friendly Room Apt. Style -UCSF/USF - San Franc...   
            4   2018-08-06                    Historic Alamo Square Victorian   

                                                         summary  \
listings_sf 0  Our bright garden unit overlooks a grassy back...   
            1                                                NaN   
            2  Nice and good public transportation.  7 minute...   
            3  Nice and good public transportation.  7 minute...   
            4  Pls email before booking.  Interior featured i...   

                                                           space  \
listings_sf 0  Newly remodeled, modern, and bright garden uni...   
            1  We live in a large Victorian house on a quiet ...   
            2  Room rental-sunny view room/sink/Wi Fi (inner ...   
            3  Room rental Sunny view Rm/Wi-Fi/TV/sink/large ...   
            4  Please send us a quick message before booking ...   

                                                     description  \
listings_sf 0  Our bright garden unit overlooks a grassy back...   
            1  We live in a large Victorian house on a quiet ...   
            2  Nice and good public transportation.  7 minute...   
            3  Nice and good public transportation.  7 minute...   
            4  Pls email before booking.  Interior featured i...   

              experiences_offered  \
listings_sf 0                none   
            1                none   
            2                none   
            3                none   
            4                none   

                                           neighborhood_overview  \
listings_sf 0  *Quiet cul de sac in friendly neighborhood *St...   
            1  I love how our neighborhood feels quiet but is...   
            2  Shopping old town, restaurants, McDonald, Whol...   
            3                                                NaN   
            4                                                NaN   

                     ...         requires_license      license  \
listings_sf 0        ...                        t  STR-0001256   
            1        ...                        t          NaN   
            2        ...                        t          NaN   
            3        ...                        t          NaN   
            4        ...                        t  STR-0000264   

              jurisdiction_names instant_bookable is_business_travel_ready  \
listings_sf 0  {"SAN FRANCISCO"}                f                        f   
            1  {"SAN FRANCISCO"}                f                        f   
            2  {"SAN FRANCISCO"}                f                        f   
            3  {"SAN FRANCISCO"}                f                        f   
            4  {"SAN FRANCISCO"}                f                        f   

                       cancellation_policy  require_guest_profile_picture  \
listings_sf 0                     moderate                              f   
            1  strict_14_with_grace_period                              f   
            2  strict_14_with_grace_period                              f   
            3  strict_14_with_grace_period                      

In [8]:
df_listings.reset_index()

,level_0,level_1,id,listing_url,scrape_id,last_scraped,name,summary,space,description,...,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,listings_sf,0,958,https://www.airbnb.com/rooms/958,2.018080e+13,2018-08-06,"Bright, Modern Garden Unit - 1BR/1B",Our bright garden unit overlooks a grassy back...,"Newly remodeled, modern, and bright garden uni...",Our bright garden unit overlooks a grassy back...,...,t,STR-0001256,"{""SAN FRANCISCO""}",f,f,moderate,f,f,1,1.38
1,listings_sf,1,5858,https://www.airbnb.com/rooms/5858,2.018080e+13,2018-08-06,Creative Sanctuary,NaN,We live in a large Victorian house on a quiet ...,We live in a large Victorian house on a quiet ...,...,t,NaN,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,1,0.99
2,listings_sf,2,7918,https://www.airbnb.com/rooms/7918,2.018080e+13,2018-08-06,A Friendly Room - UCSF/USF - San Francisco,Nice and good public transportation. 7 minute...,Room rental-sunny view room/sink/Wi Fi (inner ...,Nice and good public transportation. 7 minute...,...,t,NaN,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,9,0.16
3,listings_sf,3,8142,https://www.airbnb.com/rooms/8142,2.018080e+13,2018-08-06,Friendly Room Apt. Style -UCSF/USF - San Franc...,Nice and good public transportation. 7 minute...,Room rental Sunny view Rm/Wi-Fi/TV/sink/large ...,Nice and good public transportation. 7 minute...,...,t,NaN,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,9,0.15
4,listings_sf,4,8339,https://www.airbnb.com/rooms/8339,2.018080e+13,2018-08-06,Historic Alamo Square Victorian,Pls email before booking. Interior featured i...,Please send us a quick message before booking ...,Pls email before booking. Interior featured i...,...,t,STR-0000264,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,t,t,2,0.24
5,listings_sf,5,8567,https://www.airbnb.com/rooms/8567,2.018080e+13,2018-08-06,Lovely 2BR flat Great Location,NaN,"Fully furnished 2BR, 1BA flat in beautiful Vic...","Fully furnished 2BR, 1BA flat in beautiful Vic...",...,t,STR-0000278,"{""SAN FRANCISCO""}",f,f,moderate,f,f,1,0.27
6,listings_sf,6,8739,https://www.airbnb.com/rooms/8739,2.018080e+13,2018-08-06,"Mission Sunshine, with Private Bath","Welcome to San Francisco's Mission District, t...","Your sunny room has a queen size bed, and look...","Welcome to San Francisco's Mission District, t...",...,t,STR-0000028,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,2,5.41
7,listings_sf,7,9225,https://www.airbnb.com/rooms/9225,2.018080e+13,2018-08-06,Artful Potrero Separate Floor with Garden,A Unique Guest Suite! A Spacious Art Filled ...,"It is unique, very spacious [800 sq. ft. with...",A Unique Guest Suite! A Spacious Art Filled ...,...,t,STR-0000179,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,1,3.84
8,listings_sf,8,10251,https://www.airbnb.com/rooms/10251,2.018080e+13,2018-08-06,Victorian Suite in Inner Mission,NaN,Please read this before you book! Second floor...,Please read this before you book! Second floor...,...,t,STR-0001558,"{""SAN FRANCISCO""}",f,f,moderate,f,f,1,2.69
9,listings_sf,9,10578,https://www.airbnb.com/rooms/10578,2.018080e+13,2018-08-06,Classic Nob Hill Studio - Roof Deck,A cute studio with nice street views and lots ...,"Features: Elevator, Wood floors, TV, DVD Play...",A cute studio with nice street views and lots ...,...,t,NaN,"{""SAN FRANCISCO""}",f,f,moderate,f,t,1,0.22


In [9]:
# Creat additional column to keep track of original district

df_listings['district'] = ''

df_listings.loc['listings_sf']['district'] = 'San Francisco'
df_listings.loc['listings_ok']['district'] = 'Oakland'
df_listings.loc['listings_sc']['district'] = 'Santa Clara'
df_listings.loc['listings_scz']['district'] = 'Santa Cruz'

/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-do

In [11]:
df_district =  df_listings.loc[:, ['id', 'district']]
df_district

id       district
listings_sf  0          958  San Francisco
             1         5858  San Francisco
             2         7918  San Francisco
             3         8142  San Francisco
             4         8339  San Francisco
             5         8567  San Francisco
             6         8739  San Francisco
             7         9225  San Francisco
             8        10251  San Francisco
             9        10578  San Francisco
             10       10819  San Francisco
             11       10820  San Francisco
             12       10824  San Francisco
             13       10832  San Francisco
             14       12041  San Francisco
             15       12042  San Francisco
             16       12522  San Francisco
             17       12584  San Francisco
             18       14125  San Francisco
             19       17132  San Francisco
             20       18231  San Francisco
             21       18904  San Francisco
             22       19040  San Francisco
             23       21334  San Francisco
             24       21914  San Francisco
             25       23540  San Francisco
             26       23611  San Francisco
             27       23630  San Francisco
             28       23680  San Francisco
             29       24450  San Francisco
...                     ...            ...
listings_scz 1540  27581724     Santa Cruz
             1541  27599937     Santa Cruz
             1542  27627707     Santa Cruz
             1543  27663827     Santa Cruz
             1544  27697491     Santa Cruz
             1545  27700661     Santa Cruz
             1546  27706555     Santa Cruz
             1547  27784211     Santa Cruz
             1548  27784627     Santa Cruz
             1549  27815674     Santa Cruz
             1550  27850750     Santa Cruz
             1551  27853574     Santa Cruz
             1552  27854924     Santa Cruz
             1553  27950686     Santa Cruz
             1554  27975900     Santa Cruz
             1555  27980223     Santa Cruz
             1556  27983194     Santa Cruz
             1557  28000915     Santa Cruz
             1558  28009351     Santa Cruz
             1559  28031596     Santa Cruz
             1560  28051752     Santa Cruz
             1561  28052042     Santa Cruz
             1562  28053866     Santa Cruz
             1563  28075320     Santa Cruz
             1564  28078574     Santa Cruz
             1565  28079658     Santa Cruz
             1566  28080616     Santa Cruz
             1567  28099121     Santa Cruz
             1568  28103389     Santa Cruz
             1569  28152466     Santa Cruz

[16769 rows x 2 columns]

In [12]:
# Check Null vales within data

df_listings.isnull().sum()

id                                      0
listing_url                             0
scrape_id                               0
last_scraped                            0
name                                    2
summary                               468
space                                3489
description                           124
experiences_offered                     0
neighborhood_overview                5398
notes                                7321
transit                              5550
access                               5261
interaction                          5880
house_rules                          4658
thumbnail_url                       16769
medium_url                          16769
picture_url                             0
xl_picture_url                      16769
host_id                                 0
host_url                                0
host_name                               0
host_since                              0
host_location                     

In [13]:
df_listings.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'access', 'interaction', 'house_rules',
       'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url',
       'host_id', 'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms',

In [14]:
# Noted >90% null value in 'experience_offered', 'host_acceptance_rate', 'neighbourhood_group_cleansed', 'square_feet'
# Remove 'jurisdiction_names'
# Removed 'neighbourhood',  kept'neighbourhood_cleansed',
# Both city and 'smart location' provides city level information, and 'smart_location can be dropped
# Country code and country can be dropped, as all data located within US
# City is parent of neighbourhood_cleansed

columns_to_keep =[
       'id','host_since','host_response_time', 'host_response_rate',
        'host_is_superhost', 'host_listings_count',
       'host_total_listings_count', 
       'host_has_profile_pic', 'host_identity_verified', 
        'neighbourhood_cleansed',
       'city', 
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type','amenities', 
       'price', 'security_deposit',
       'cleaning_fee', 'guests_included',  'minimum_nights','number_of_reviews',
       'first_review', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'jurisdiction_names', 'instant_bookable',
       'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification',
       'calculated_host_listings_count', 'reviews_per_month', 'district']

In [15]:
df_smaller_listings = df_listings[columns_to_keep]

In [16]:
df_smaller_listings

id host_since  host_response_time host_response_rate  \
listings_sf  0          958 2008-07-31      within an hour                92%   
             1         5858 2009-03-02      within an hour               100%   
             2         7918 2009-06-17  within a few hours               100%   
             3         8142 2009-06-17  within a few hours               100%   
             4         8339 2009-07-02      within an hour               100%   
             5         8567 2009-07-14        within a day                80%   
             6         8739 2009-01-27      within an hour               100%   
             7         9225 2009-08-04      within an hour               100%   
             8        10251 2009-08-31      within an hour               100%   
             9        10578 2009-09-08      within an hour               100%   
             10       10819 2009-09-15        within a day                99%   
             11       10820 2009-09-15        within a day                99%   
             12       10824 2009-09-15        within a day                99%   
             13       10832 2009-09-15        within a day                99%   
             14       12041 2009-06-17  within a few hours               100%   
             15       12042 2009-06-17  within a few hours               100%   
             16       12522 2009-05-22      within an hour               100%   
             17       12584 2009-10-29  within a few hours               100%   
             18       14125 2009-11-18  within a few hours               100%   
             19       17132 2009-10-04      within an hour               100%   
             20       18231 2010-01-13  within a few hours               100%   
             21       18904 2010-01-21  within a few hours               100%   
             22       19040 2010-01-22  within a few hours               100%   
             23       21334 2010-02-15      within an hour               100%   
             24       21914 2009-06-17  within a few hours               100%   
             25       23540 2010-03-13      within an hour               100%   
             26       23611 2009-05-25      within an hour               100%   
             27       23630 2010-03-14      within an hour               100%   
             28       23680 2010-03-15  within a few hours               100%   
             29       24450 2010-03-25      within an hour               100%   
...                     ...        ...                 ...                ...   
listings_scz 1540  27581724 2018-06-29                 NaN                NaN   
             1541  27599937 2011-01-10      within an hour               100%   
             1542  27627707 2014-11-28                 NaN                NaN   
             1543  27663827 2014-02-25      within an hour               100%   
             1544  27697491 2018-08-12      within an hour                80%   
             1545  27700661 2017-10-21      within an hour               100%   
             1546  27706555 2015-12-26  within a few hours               100%   
             1547  27784211 2017-10-12      within an hour               100%   
             1548  27784627 2014-08-21      within an hour               100%   
             1549  27815674 2016-07-26      within an hour               100%   
             1550  27850750 2018-08-18      within an hour               100%   
             1551  27853574 2018-08-18      within an hour               100%   
             1552  27854924 2018-08-18      within an hour               100%   
             1553  27950686 2014-09-29                 NaN                NaN   
             1554  27975900 2017-11-10        within a day               100%   
             1555  27980223 2017-11-10        within a day               100%   
             1556  27983194 2014-08-02  within a few hours                54%   
             1557  28000915 2017-01-16      within an ho

In [17]:
# This is to onvert strings with percentage sign to float

def percent_to_float(string):
    """
    Convert strings with percentage sign to float
    """
    percentage = float(str(string).strip('%'))/100
    return percentage

In [18]:
# Convert host_response_rate to float

df_smaller_listings['host_response_rate'] = df_smaller_listings['host_response_rate']  \
                                            .apply(lambda x: percent_to_float(x))

/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
def currency_to_float(string):
    """
    Convert currency to float
    """
    float_ = float(str(string).strip('$').replace(',',''))
    return float_

In [20]:
# Convert currency features to float

df_smaller_listings['price'] = df_smaller_listings['price'].apply(lambda x: currency_to_float(x))
df_smaller_listings['security_deposit'] = df_smaller_listings['security_deposit'].apply(lambda x: currency_to_float(x))
df_smaller_listings['cleaning_fee'] = df_smaller_listings['cleaning_fee'].apply(lambda x: currency_to_float(x))

/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

In [21]:
df_smaller_listings.shape

(16769, 43)

In [22]:
df_smaller_listings.price.describe(percentiles=[.05, .10, .25, .5, .75, .90, .95])

count    16769.000000
mean       188.023555
std        274.564274
min          0.000000
5%          41.000000
10%         52.000000
25%         79.000000
50%        125.000000
75%        201.000000
90%        350.000000
95%        499.000000
max      10000.000000
Name: price, dtype: float64

In [23]:
# Remove outliers. Use 5% and 95% percentiles as cutoff

df_smaller_listings = df_smaller_listings[(df_smaller_listings['price']<=500) 
                                         & (df_smaller_listings['price']>=40)]

In [24]:
df_smaller_listings.shape

(15379, 43)

In [25]:
# Remove special characters within Amenities

df_smaller_listings['jurisdiction_names'] = df_smaller_listings['jurisdiction_names'].str.replace('[^\w\s,]', '')
df_smaller_listings['amenities'] = df_smaller_listings['amenities'].str.replace('[^\w\s,]', '')

/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [26]:
df_smaller_listings.amenities.head()

listings_sf  0    TV,Cable TV,Internet,Wifi,Kitchen,Pets live on...
             1    Internet,Wifi,Kitchen,Heating,Familykid friend...
             2    TV,Internet,Wifi,Kitchen,Free street parking,H...
             3    TV,Internet,Wifi,Kitchen,Free street parking,H...
             5    TV,Cable TV,Internet,Wifi,Kitchen,Free parking...
Name: amenities, dtype: object

In [27]:
df_amenities = df_smaller_listings  \
               .amenities  \
               .str.get_dummies(sep = ",")  \
               .add_prefix('amenities_')

In [28]:
df_amenities.head()

amenities_ toilet  amenities_24hour checkin  \
listings_sf 0                  0                         1   
            1                  0                         0   
            2                  0                         0   
            3                  0                         0   
            5                  0                         0   

               amenities_Accessibleheight bed  \
listings_sf 0                               0   
            1                               0   
            2                               0   
            3                               0   
            5                               0   

               amenities_Accessibleheight toilet  amenities_Air conditioning  \
listings_sf 0                                  0                           0   
            1                                  0                           0   
            2                                  0                           0   
            3                                  0                           0   
            5                                  0                           0   

               amenities_Air purifier  amenities_Amazon Echo  \
listings_sf 0                       0                      0   
            1                       0                      0   
            2                       0                      0   
            3                       0                      0   
            5                       0                      0   

               amenities_BBQ grill  amenities_Baby bath  \
listings_sf 0                    0                    0   
            1                    0                    0   
            2                    0                    0   
            3                    0                    0   
            5                    1                    0   

               amenities_Baby monitor  \
listings_sf 0                       0   
            1                       0   
            2                       0   
            3                       0   
            5                       0   

                                      ...                          \
listings_sf 0                         ...                           
            1                         ...                           
            2                         ...                           
            3                         ...                           
            5                         ...                           

               amenities_Wide clearance to bed  \
listings_sf 0                                0   
            1                                0   
            2                                0   
            3                                0   
            5                                0   

               amenities_Wide clearance to shower  amenities_Wide doorway  \
listings_sf 0                                   0                       0   
            1                                   0                       0   
            2                                   0                       0   
            3                                   0                       0   
            5                                   0                       0   

               amenities_Wide entryway  amenities_Wide hallway clearance  \
listings_sf 0                        0                                 0   
            1                        0                                 0   
            2                        0                                 0   
            3                        0                                 0   
            5                        0                                 0   

               amenities_Wifi  amenities_Window guards  amenities_Wine cooler  \
listings_sf 0               1                        0                      0   
            1               1                        0                      0   


In [29]:
# There is a long list of amenities. This is to filter down to common amenities.

common_amenities=[]

for column in df_amenities.columns:
    value = df_amenities[column].quantile(0.75)
    if value>0:
        common_amenities.append(column)

common_amenities

['amenities_Air conditioning',
 'amenities_Bed linens',
 'amenities_Cable TV',
 'amenities_Carbon monoxide detector',
 'amenities_Coffee maker',
 'amenities_Cooking basics',
 'amenities_Dishes and silverware',
 'amenities_Dryer',
 'amenities_Essentials',
 'amenities_Extra pillows and blankets',
 'amenities_Familykid friendly',
 'amenities_Fire extinguisher',
 'amenities_First aid kit',
 'amenities_Free parking on premises',
 'amenities_Free street parking',
 'amenities_Hair dryer',
 'amenities_Hangers',
 'amenities_Heating',
 'amenities_Hot water',
 'amenities_Internet',
 'amenities_Iron',
 'amenities_Kitchen',
 'amenities_Laptop friendly workspace',
 'amenities_Lock on bedroom door',
 'amenities_Long term stays allowed',
 'amenities_Microwave',
 'amenities_Oven',
 'amenities_Private entrance',
 'amenities_Refrigerator',
 'amenities_Self checkin',
 'amenities_Shampoo',
 'amenities_Smoke detector',
 'amenities_Stove',
 'amenities_TV',
 'amenities_Washer',
 'amenities_Wifi']

In [30]:
df_common_amenities = df_amenities[common_amenities]
df_common_amenities.head()

amenities_Air conditioning  amenities_Bed linens  \
listings_sf 0                           0                     0   
            1                           0                     0   
            2                           0                     0   
            3                           0                     0   
            5                           0                     0   

               amenities_Cable TV  amenities_Carbon monoxide detector  \
listings_sf 0                   1                                   1   
            1                   0                                   0   
            2                   0                                   1   
            3                   0                                   1   
            5                   1                                   1   

               amenities_Coffee maker  amenities_Cooking basics  \
listings_sf 0                       0                         0   
            1                       0                         0   
            2                       0                         0   
            3                       0                         0   
            5                       1                         1   

               amenities_Dishes and silverware  amenities_Dryer  \
listings_sf 0                                0                1   
            1                                0                1   
            2                                0                1   
            3                                0                1   
            5                                1                1   

               amenities_Essentials  amenities_Extra pillows and blankets  \
listings_sf 0                     1                                     0   
            1                     1                                     0   
            2                     0                                     0   
            3                     0                                     0   
            5                     1                                     0   

                    ...        amenities_Oven  amenities_Private entrance  \
listings_sf 0       ...                     0                           1   
            1       ...                     0                           1   
            2       ...                     0                           1   
            3       ...                     0                           1   
            5       ...                     1                           0   

               amenities_Refrigerator  amenities_Self checkin  \
listings_sf 0                       0                       1   
            1                       0                       0   
            2                       0                       0   
            3                       0                       0   
            5                       1                       0   

               amenities_Shampoo  amenities_Smoke detector  amenities_Stove  \
listings_sf 0                  1                         1                0   
            1                  1                         1                0   
            2                  0                         1                0   
            3                  0                         1                0   
            5                  1                         1                1   

               amenities_TV  amenities_Washer  amenities_Wifi  
listings_sf 0             1                 1               1  
            1             0                 1               1  
            2             1                 1               1  
            3             1                 1               1  
            5             1                 1               1  

[5 rows x 36 columns]

In [31]:
df_listings_refined = pd.concat([df_smaller_listings, df_common_amenities], axis=1)

In [32]:
df_listings_refined.shape

(15379, 79)

In [33]:
df_listings_refined.drop('amenities', axis=1, inplace=True)

In [34]:
df_listings_refined.columns

Index(['id', 'host_since', 'host_response_time', 'host_response_rate',
       'host_is_superhost', 'host_listings_count', 'host_total_listings_count',
       'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'city', 'is_location_exact', 'property_type',
       'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'bed_type', 'price', 'security_deposit', 'cleaning_fee',
       'guests_included', 'minimum_nights', 'number_of_reviews',
       'first_review', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'jurisdiction_names', 'instant_bookable',
       'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification',
       'calculated_host_listings_count', 'reviews_per_month', 'district',
       'amenities_Air conditioning', 

In [35]:
# This is to convert timestamp feature to duration

def duration(starting_time):
    """
    Calculate duration or number of years between current year and input years
    """
    starting_year = starting_time.year
    current_year = datetime.now().year
    duration = current_year - starting_year + 1
    
    return duration

In [36]:
# Calculate durations for timestamp columns

df_listings_refined['host_yrs'] = df_listings_refined['host_since'].apply(lambda x: duration(x))
df_listings_refined['yrs_since_first_review'] = df_listings_refined['first_review'].apply(lambda x: duration(x))

In [37]:
# Drop original and un-used timestamp features

time_stamps =['host_since', 'first_review']
df_listings_refined.drop(time_stamps, axis=1, inplace=True)

In [ ]:
df_listings_refined.isnull().sum()

In [ ]:
# Filled null values with average for security deposits and cleaning fee

df_listings_refined['security_deposit'].fillna(
    value=df_listings_refined['security_deposit'].mean(),
    inplace=True)

df_listings_refined['cleaning_fee'].fillna(
    value=df_listings_refined['cleaning_fee'].mean(),
    inplace=True)

In [ ]:
# Security deposit can vary in big range. Convert it to boolean varialbes

df_listings_refined['security_deposit'] = (df_listings_refined['security_deposit']>0).astype(int)

In [ ]:
df_listings_refined.describe(include='all')

In [ ]:
df_listings_refined.review_scores_rating.describe(include='all')

In [ ]:
df_listings_refined.review_scores_accuracy.describe(include='all')

In [ ]:
# Those varialbes have big range. Need to narrow down range to enable better modeling (take log10)


df_listings_refined['cleaning_fee'] = df_listings_refined['cleaning_fee'].apply(lambda x: np.log10(x) if x>0 else 0)
df_listings_refined['review_scores_rating_log10'] = df_listings_refined['review_scores_rating'].apply(lambda x: np.log10(x) if x>0 else 0)
df_listings_refined['review_scores_accuracy_log10'] = df_listings_refined['review_scores_accuracy'].apply(lambda x: np.log10(x) if x>0 else 0)
df_listings_refined['review_scores_checkin_log10'] = df_listings_refined['review_scores_checkin'].apply(lambda x: np.log10(x) if x>0 else 0)
df_listings_refined['review_scores_cleanliness_log10'] = df_listings_refined['review_scores_cleanliness'].apply(lambda x: np.log10(x) if x>0 else 0)
df_listings_refined['review_scores_communication_log10'] = df_listings_refined['review_scores_communication'].apply(lambda x: np.log10(x) if x>0 else 0)
df_listings_refined['review_scores_location_log10'] = df_listings_refined['review_scores_location'].apply(lambda x: np.log10(x) if x>0 else 0)
df_listings_refined['review_scores_value_log10'] = df_listings_refined['review_scores_value'].apply(lambda x: np.log10(x) if x>0 else 0)


In [ ]:
# Drop original ratings

review_scores =['review_scores_rating','review_scores_accuracy',
                'review_scores_checkin','review_scores_cleanliness', 
                'review_scores_communication', 'review_scores_location', 
                'review_scores_value']
df_listings_refined.drop(review_scores, axis=1, inplace=True)

In [ ]:
# Minimum nights can vary in big range. Convert it to boolean varialbes

df_listings_refined['minimum_nights'] = (df_listings_refined['minimum_nights']>1).astype(int)
df_listings_refined.minimum_nights.head()

In [ ]:
df_listings_refined.isnull().sum()

In [ ]:
# Drop rest of Null values

df_listings_cleaned = df_listings_refined.dropna()

In [ ]:
df_listings_cleaned.shape

In [ ]:
df_listings_cleaned.isnull().sum()

In [ ]:
df_listings_cleaned.columns

In [ ]:
df_jurisdiction = df_listings_cleaned.loc[:, ['id', 'jurisdiction_names']]

In [ ]:
# select non-numeric variables and create dummies

categorical_variables = df_listings_cleaned.select_dtypes(include=['object']).columns
df_listings_cleaned[categorical_variables].head()

In [ ]:
categorical_variables.shape

In [ ]:
df_categorical.columns

In [ ]:
df_model = pd.get_dummies(data=df_listings_cleaned,columns=categorical_variables)

In [ ]:
df_model.shape

In [ ]:
df_model.columns

In [ ]:
# Create correlation matrix
corr_matrix = df_model.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.90
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

In [ ]:
to_drop

In [ ]:
# Drop features 
df_model.drop(to_drop, axis=1, inplace=True)

In [ ]:
df_model.dropna(inplace=True)

In [ ]:
df_model.columns

In [ ]:
# Store as csv file for feature investigation

csv_path = '/Users/xzhou/github/project_archives/files_airbnb/columns_check.csv'

df_model.to_csv(csv_path)

In [ ]:
pkl_file = '/Users/xzhou/github/project_archives/files_airbnb/{}.csv'

pkl_listings = pkl_file.format('all_listings')

df_model.to_pickle(pkl_listings)

In [ ]:
# Perform initial modelings to get a sense of performance, and for future selection

X = df_model.drop(['id','price'], axis=1)
y = df_model['price']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3, random_state=42)

In [ ]:
ss = StandardScaler()
X_train_trans = ss.fit_transform(X_train)
X_test_trans = ss.transform (X_test)

In [ ]:
lr = linear_model.LinearRegression()
lr.fit(X_train_trans, y_train)

In [ ]:
lr.score(X_test_trans, y_test)

In [ ]:
y_predict = lr.predict(X_test_trans)

In [ ]:
r2_score(y_predict, y_test)

In [ ]:
mean_squared_error(y_predict, y_test)

In [ ]:
model = sm.OLS(y_train, sm.add_constant(X_train_trans))
fit = model.fit()
fit.summary()

In [ ]:
X_train_trans

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=500, 
                               criterion='mse', 
                               random_state=3, 
                               n_jobs=-1)
rf.fit(X_train_trans, y_train)
y_train_pred = rf.predict(X_train_trans)
y_test_pred = rf.predict(X_test_trans)

print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

In [ ]:
feature_labels = X.columns
feature_tuples = zip (feature_labels, rf.feature_importances_)
feature_lists = [list(feature_tuple) for feature_tuple in feature_tuples]

In [ ]:
feature_lists.sort(key=lambda x: abs(x[1]), reverse=True)

In [ ]:
feature_lists

In [ ]:
features_to_keep=[]

for (feature, importance) in feature_lists:
    if importance>=0.003:
        features_to_keep.append(feature)
    else:
        break

features_to_keep

In [ ]:
len(features_to_keep)

In [ ]:
no_features_kept = len(features_to_keep)
features_w_importance = feature_lists[0: no_features_kept]
features_w_importance

In [ ]:
X_updated = df_model[features_to_keep]
y_updated = df_model['price']

In [ ]:
X_updated_train, X_updated_test, y_updated_train, y_updated_test = train_test_split(
    X_updated, y_updated, test_size =0.3, random_state=42)

In [ ]:
ssX2 = StandardScaler()
X_updated_train_trans = ssX2.fit_transform(X_updated_train)
X_updated_test_trans = ssX2.transform (X_updated_test)

In [ ]:
rf_updated = RandomForestRegressor(n_estimators=500, 
                                   criterion='mse', 
                                   random_state=3, 
                                   n_jobs=-1)
rf_updated.fit(X_updated_train_trans, y_updated_train)
y_updated_train_pred = rf_updated.predict(X_updated_train_trans)
y_updated_test_pred = rf_updated.predict(X_updated_test_trans)

print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_updated_train, y_updated_train_pred),
        mean_squared_error(y_updated_test, y_updated_test_pred)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_updated_train, y_updated_train_pred),
        r2_score(y_updated_test, y_updated_test_pred)))

In [ ]:
# Store selected feautre for future use

pkl_file = '/Users/xzhou/github/project_archives/files_airbnb/{}.csv'

pkl_listings = pkl_file.format('all_listings')

df_test_model.to_pickle(pkl_listings)